In [9]:
#Install Pytorch
#import sys
#!{sys.executable} -m pip install torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2+cu118 --index-url https://download.pytorch.org/whl/cu118
#!{sys.executable} -m pip install tensorboard==2.9.1

In [10]:
'''
import torch
import torchvision
print(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")
if not torch.__version__.startswith("2.2"):
    print("you are using an another version of PyTorch. We expect PyTorch 2.2. You may continue using your version but it"
          " might cause dependency and compatibility issues.")
if not torchvision.__version__.startswith("0.17"):
    print("you are using an another version of torchvision. We expect torchvision 0.17. You can continue with your version but it"
          " might cause dependency and compatibility issues.")
'''

'\nimport torch\nimport torchvision\nprint(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")\nif not torch.__version__.startswith("2.2"):\n    print("you are using an another version of PyTorch. We expect PyTorch 2.2. You may continue using your version but it"\n          " might cause dependency and compatibility issues.")\nif not torchvision.__version__.startswith("0.17"):\n    print("you are using an another version of torchvision. We expect torchvision 0.17. You can continue with your version but it"\n          " might cause dependency and compatibility issues.")\n'

In [19]:
import sys
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import requests

from models import tokenizer, embedding, transformer

import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

%load_ext autoreload
%autoreload 2
%matplotlib inline

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text

with open("data/tiny_shakespeare.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Tiny Shakespeare downloaded! File size:", len(text), "characters")

Tiny Shakespeare downloaded! File size: 1115394 characters


In [ ]:
#tokenizer
with open("data/tiny_shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

tok = tokenizer.CharTokenizer(text)
print(len(tok.chars), "unique chars")

#Sample Test

'''
sample = "ROMEO:"
ids = tok.encode(sample)
print("Encoded:", ids)
print("Decoded:", tok.decode(ids))
'''

65 unique chars


'\nsample = "ROMEO:"\nids = tok.encode(sample)\nprint("Encoded:", ids)\nprint("Decoded:", tok.decode(ids))\n'

In [ ]:
ids = tok.encode(text)
data = torch.tensor(ids, dtype=torch.long)

# Split by 6:2:2
n = len(data)
n_train = int(0.6 * n)
n_val = int(0.2 * n)
n_test = n - n_train - n_val

train_data = data[:n_train]
val_data = data[n_train:n_train + n_val]
test_data = data[n_train + n_val:]

print(f"Total tokens: {n:,}")
print(f"Train: {len(train_data):,}, Val: {len(val_data):,}, Test: {len(test_data):,}")

Total tokens: 1,115,394
Train: 669,236, Val: 223,078, Test: 223,080


In [17]:
vocab_size = 65 
d_model = 128
block_size = 256

embed = embedding.TokenAndPositionEmbedding(vocab_size, d_model, block_size)

x = torch.randint(0, vocab_size, (4, 20))

out = embed(x)

print("Input shape:", x.shape)
print("Output shape:", out.shape)

Input shape: torch.Size([4, 20])
Output shape: torch.Size([4, 20, 128])


In [20]:
torch.manual_seed(0)
B, T, C = 2, 16, 128
H, D = 4, C // 4
block_size = 32
dropout = 0.1

x = torch.randn(B, T, C)

# Only Attention
attn = transformer.CausalSelfAttention(d_model=C, n_heads=H, dropout=dropout, block_size=block_size)
y = attn(x)  # (B, T, C)
print("attn out:", y.shape)

# Full Block
block = transformer.TransformerBlock(d_model=C, n_heads=H, d_ff=4*C, dropout=dropout, block_size=block_size)
z = block(x)
print("block out:", z.shape)

attn out: torch.Size([2, 16, 128])
block out: torch.Size([2, 16, 128])
